# <center>Extractive Text Summarization Using Python and Gensim</center>

__<center>by Tauno Tanilas - 2020</center>__

## 1. Introduction

<img src="https://st2.ning.com/topology/rest/1.0/file/get/502818197?profile=original" width="500" align="left">

## 2. Data

### 2.1. Data Acquiring

Import Required Libraries.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [7]:
#!pip install gensim

In [6]:
import urllib.request as ur
import bs4 as bs
import pandas as pd
import nltk
import re
import gensim
from gensim.summarization import summarize

#nltk.download('punkt') # one time execution
#nltk.download('stopwords')

Read the Data

In [9]:
def scrape_data(url):
    scraped_data = ur.urlopen(url)
    article = scraped_data.read()
    
    parsed_article = bs.BeautifulSoup(article,'lxml')
    paragraphs = parsed_article.find_all('p')
    
    article_text = ""
    
    for p in paragraphs:
        article_text += p.text
    return article_text

In [10]:
article_topic = ['Bill_Gates', 'Steve_Jobs', 'Jeff_Bezos']
article_url = ['https://en.wikipedia.org/wiki/Bill_Gates',
               'https://en.wikipedia.org/wiki/Steve_jobs',
               'https://en.wikipedia.org/wiki/Jeff_Bezos']

article_df_cn = ['article_topic', 'article_text', 'article_url']
article_df = pd.DataFrame(columns=article_df_cn)

for u_idx, art_url in enumerate(article_url):
    article_df.at[u_idx, article_df_cn[0]] = article_topic[u_idx]
    article_df.at[u_idx, article_df_cn[1]] = scrape_data(art_url)
    article_df.at[u_idx, article_df_cn[2]] = article_url[u_idx]

article_df

,article_topic,article_text,article_url
0,Bill_Gates,"\nWilliam Henry Gates III (born October 28, 1...",https://en.wikipedia.org/wiki/Bill_Gates
1,Steve_Jobs,"\n\nSteven Paul Jobs (/dʒɒbz/; February 24, 19...",https://en.wikipedia.org/wiki/Steve_jobs
2,Jeff_Bezos,\nJeffrey Preston Bezos (/ˈbeɪzoʊs/;[a][2] né ...,https://en.wikipedia.org/wiki/Jeff_Bezos


Inspect the Data

In [12]:
article_df['article_text'][0]

'\nWilliam Henry Gates III  (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist. He is best known as the co-founder of Microsoft Corporation.[2][3] During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014. He is one of the best-known entrepreneurs and pioneers of the microcomputer revolution of the 1970s and 1980s.\nBorn and raised in Seattle, Washington, Gates co-founded Microsoft with childhood friend Paul Allen in 1975, in Albuquerque, New Mexico; it went on to become the world\'s largest personal computer software company.[4][a] Gates led the company as chairman and CEO until stepping down as CEO in January 2000, but he remained chairman and became chief software architect.[7] During the late 1990s, Gates had been criticized for his business tactics, which have been considered ant

### 2.2. Data Cleaning and Preparation

Split Text into Sentences.

In [13]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in article_df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [14]:
sentences[:5]

['\nWilliam Henry Gates III  (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist.',
 'He is best known as the co-founder of Microsoft Corporation.',
 '[2][3] During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014.',
 'He is one of the best-known entrepreneurs and pioneers of the microcomputer revolution of the 1970s and 1980s.',
 "Born and raised in Seattle, Washington, Gates co-founded Microsoft with childhood friend Paul Allen in 1975, in Albuquerque, New Mexico; it went on to become the world's largest personal computer software company."]

In [15]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [16]:
clean_sentences[:5]

[' william henry gates iii   born october           is an american business magnate  software developer  investor  and philanthropist ',
 'he is best known as the co founder of microsoft corporation ',
 '       during his career at microsoft  gates held the positions of chairman  chief executive officer  ceo   president and chief software architect  while also being the largest individual shareholder until may      ',
 'he is one of the best known entrepreneurs and pioneers of the microcomputer revolution of the     s and     s ',
 'born and raised in seattle  washington  gates co founded microsoft with childhood friend paul allen in       in albuquerque  new mexico  it went on to become the world s largest personal computer software company ']

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [18]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [19]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [20]:
clean_sentences[:5]

['william henry gates iii born october american business magnate software developer investor philanthropist',
 'best known co founder microsoft corporation',
 'career microsoft gates held positions chairman chief executive officer ceo president chief software architect also largest individual shareholder may',
 'one best known entrepreneurs pioneers microcomputer revolution',
 'born raised seattle washington gates co founded microsoft childhood friend paul allen albuquerque new mexico went become world largest personal computer software company']

## 3. TextRank Algorithm

### 3.1. Vector Representation of Sentences

In [97]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

### 3.2. Similarity Matrix Preparation

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

### 3.3. Applying PageRank Algorithm

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

### 3.4. Summary Extraction

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
# Extract top 10 sentences as the summary
for i in range(10):
    print(ranked_sentences[i][1])

Gensim

In [24]:
summary = summarize(article_df['article_text'][0], ratio=0.1)

In [25]:
summary

'William Henry Gates III  (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist.\nHe is best known as the co-founder of Microsoft Corporation.[2][3] During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014.\nBorn and raised in Seattle, Washington, Gates co-founded Microsoft with childhood friend Paul Allen in 1975, in Albuquerque, New Mexico; it went on to become the world\'s largest personal computer software company.[4][a] Gates led the company as chairman and CEO until stepping down as CEO in January 2000, but he remained chairman and became chief software architect.[7] During the late 1990s, Gates had been criticized for his business tactics, which have been considered anti-competitive.\nThis opinion has been upheld by numerous court rulings.[8] In June 2006, Gates announced that he